# 实测技术分析

### 本段代码利用quantOS系统分析了几个常见技术信号有效性，我们选取了两个技术形态：三根阳
线，金叉死叉，以及一个技术指标：跳空高开。大致步骤如下：


1.首先通过`add_formula`功能将信号用公式表达并计算，再加入到`DataView`中

2.利用信号测试模块`jaqs.research.SignalDigger`分析信号有效性

## 1. 准备工作

### 1.1 设置参数

In [3]:
from __future__ import unicode_literals

import matplotlib.pyplot as plt
%matplotlib inline  

import numpy as np
import pandas as pd

import seaborn as sns

from jaqs.data import DataView
from jaqs.data import RemoteDataService
from jaqs.research import SignalDigger
import jaqs.util as jutil

In [4]:
# 设置服务器地址、用户名密码

# 如果没有使用quantos金融终端，请自行替换phone,token内容
import os
phone = os.environ.get("QUANTOS_USER")
token = os.environ.get("QUANTOS_TOKEN")

data_config = {
  "timeout": 1800,
  "remote.data.address": "tcp://data.tushare.org:8910",
  "remote.data.username":  phone,
  "remote.data.password":  token}

dataview_folder = 'technical_analysis'

### 1.2 初始化DataView

In [5]:
# 选股范围和基准设为沪深300指数
ds = RemoteDataService()
ds.init_from_config(data_config)
dv = DataView()

props = {'start_date': 20130101, 'end_date': 20171031, 'universe': '000300.SH',
         'fields': 'volume,turnover',
         'freq': 1}

dv.init_from_config(props, ds)
dv.prepare_data()

dv.save_dataview(dataview_folder)


Begin: DataApi login 18612562791@tcp://data.tushare.org:8910
    login success 

Initialize config success.
Query data...
Query data - query...
NOTE: price adjust method is [post adjust]
201
403
605
807
1009
1211
201
403
605
807
1009
1211
Query data - daily fields prepared.
Query instrument info...
Query adj_factor...
Query benchmark...
Query benchmar member info...
Data has been successfully prepared.

Store data...
Dataview has been successfully saved to:
C:\Users\jfang\AppData\Roaming\QuantosFinanceTerminal\workspace\__lectures\__primaryschool\technical_analysis

You can load it with load_dataview('C:\Users\jfang\AppData\Roaming\QuantosFinanceTerminal\workspace\__lectures\__primaryschool\technical_analysis')


### 1.3 检查股票是否出现停牌、调处成份股和达到涨跌幅限制的情况，分析时将会过滤掉

In [6]:
# 判断个股当前是否达到涨跌停限制：达到 = True， 未达到 = False
dv.add_formula('mask_limit_reached', 'Abs((open - Delay(close, 1)) / Delay(close, 1)) > 0.095', is_quarterly=False)

# 判断个股当期是否为成份股：是成份股 = True， 非成份股 = False
dv.add_formula('mask_index_member', '(index_member == 0)', is_quarterly=False)

# 判断个股当前是否处于停牌状态
trade_status = dv.get_ts('trade_status')
mask_sus = trade_status == u'停牌'
dv.append_df(mask_sus, 'mask_sus', is_quarterly=False)

D:\QuantosTerminal\python\lib\site-packages\jaqs\data\py_expression_eval.py:472: RuntimeWarning: invalid value encountered in greater
  res = arr > brr


In [7]:
# 将以上三个条件整合
mask_limit_reached = dv.get_ts('mask_limit_reached')
mask_index_member = dv.get_ts('mask_index_member')
mask_sus = dv.get_ts('mask_sus')
mask_all = np.logical_or(mask_sus, np.logical_or(mask_index_member, mask_limit_reached))

## 取出股价序列和指数价格序列 用于计算超额收益
这里使用后复权价格

In [8]:
price = dv.get_ts('close_adj')
price_bench = dv.data_benchmark

## 2. 技术指标一：金叉死叉 

### 2.1 定义金叉死叉，并通过add_formula添加到dataview中

In [9]:
# 定义快速均线(5天)和慢速均线(15天)
dv.add_formula('fast_MA', 'Ts_Mean(close_adj, 5)', is_quarterly=False)
dv.add_formula('slow_MA', 'Ts_Mean(close_adj, 15)', is_quarterly=False)

# 定义金叉
factor_formula = '(Delay(fast_MA, 1) < Delay(slow_MA, 1) ) && (fast_MA > slow_MA)'
dv.add_formula('golden_cross', factor_formula, is_quarterly=False)

# 定义死叉
factor_formula = '(Delay(fast_MA, 1) > Delay(slow_MA, 1) ) && (fast_MA < slow_MA)'
dv.add_formula('dark_cross', factor_formula, is_quarterly=False)

D:\QuantosTerminal\python\lib\site-packages\jaqs\data\py_expression_eval.py:608: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=5,center=False).mean()
  return pd.rolling_mean(x, n)
D:\QuantosTerminal\python\lib\site-packages\jaqs\data\py_expression_eval.py:608: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=15,center=False).mean()
  return pd.rolling_mean(x, n)
D:\QuantosTerminal\python\lib\site-packages\jaqs\data\py_expression_eval.py:481: RuntimeWarning: invalid value encountered in less
  res = arr < brr
D:\QuantosTerminal\python\lib\site-packages\jaqs\data\py_expression_eval.py:472: RuntimeWarning: invalid value encountered in greater
  res = arr > brr


### 2.2 分析死叉信号

In [ ]:
signal = dv.get_ts('dark_cross').shift(1, axis=0)  # avoid look-ahead bias
obj = SignalDigger(output_folder='technical_analysis', output_format='plot')
df_all, df_events, df_stats = obj.create_binary_event_report(signal, price, mask_all, price_bench,
                                                             periods=[1, 5, 10, 20], group_by='year')

Nan Data Count (should be zero) : 0;  Percentage of effective data: 94%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 94%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 94%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 93%


In [ ]:
df_stats

In [ ]:
# 以等权收益作为基准，计算超额收益
sns.barplot(x='Period', y=0, hue='trade_date',
            data=(df_stats['Annu. Ret.'] - df_stats['Annu. Ret. (all samp)']).reset_index())

### 2.3 分析金叉信号

In [ ]:
signal = dv.get_ts('golden_cross')  # avoid look-ahead bias
obj = SignalDigger(output_folder=dataview_folder, output_format='plot')
df_all, df_events, df_stats = obj.create_binary_event_report(signal, price, mask_all, price_bench,
                                                             periods=[1, 5, 10, 20], group_by='year')

In [ ]:
df_stats

In [ ]:
# 以等权收益作为基准，计算超额收益
sns.barplot(x='Period', y=0, hue='trade_date',
            data=(df_stats['Annu. Ret.'] - df_stats['Annu. Ret. (all samp)']).reset_index())

## 3. 技术指标二：三根阳线改变预期

### 3.1 定义三根阳线，并通过add_formula加入dataview中

In [ ]:
# 定义单根阳线
formula = '(close_adj > open_adj) && ( ( (close_adj - open_adj)/(high_adj - low_adj) ) >= 0.7)'
dv.add_formula('UP', formula, is_quarterly = False)
# 定义三根阳线
formula = 'UP && Delay(UP, 1) && Delay(UP, 2)'
dv.add_formula('UP3', formula, is_quarterly = False)

### 3.2 分析三根阳线信号

In [ ]:
signal = dv.get_ts('UP3')  # avoid look-ahead bias
price = dv.get_ts('close_adj')
obj = SignalDigger(output_folder=dataview_folder, output_format='plot')
df_all, df_events, df_stats = obj.create_binary_event_report(signal, price, mask_all, price_bench,
                                                             periods=[1, 5, 10, 20], group_by='year')

In [ ]:
df_stats

In [ ]:
# 以等权收益作为基准，计算超额收益
sns.barplot(x='Period', y=0, hue='trade_date',
            data=(df_stats['Annu. Ret.'] - df_stats['Annu. Ret. (all samp)']).reset_index())

## 4. 技术指标三：跳空高开 / 低开

### 4.1 定义为今开减昨收，并通过add_formula加入dataview中

In [ ]:
# 定义跳空高开
formula = 'open_adj / Delay(close_adj, 1)'
dv.add_formula('open_jump', formula, is_quarterly=False) # good

### 4.2 分析跳空高开信号 

In [ ]:
signal = dv.get_ts('open_jump')  # avoid look-ahead bias
my_period = 5  # 5日调仓，一年10% cost，因而10%一下收益无法战胜成本

obj = SignalDigger(output_folder=dataview_folder, output_format='plot')
obj.process_signal_before_analysis(signal, price=price,
                                   mask=mask_all,
                                   n_quantiles=5, period=my_period,
                                   benchmark_price=price_bench,
                                   )
res = obj.create_full_report()

#没有纯空头收益，因为信号总是正的